# Missing users 9h-11h

In [5]:
import pandas as pd
import numpy as np
import os
import itertools
import random
import utm

In [10]:
input_path1 = os.getcwd() + "/OUTPUTS_9h_11h/path.csv"
input_path2 = os.getcwd() + "/INPUTS/test_9h_11h.csv"
Output = pd.read_csv(input_path1, delimiter = ';')
Input = pd.read_csv(input_path2, delimiter = ';')
listID_in = Input['ID'].tolist()
listID_out = Output['ID'].tolist()

In [11]:
def diff(l1, l2):   # return a list of lost users
    difference = list(set(l1).symmetric_difference(set(l2)))
    return difference

In [107]:
Input

,ID,DEPARTURE,ORIGIN,DESTINATION
0,1019364,09:00:00,624582.3745792772 5804534.655578803,619360.0571446231 5805160.322357742
1,958538,09:00:00,612373.9501705638 5801667.450917384,632339.2294622243 5797212.880052918
2,1015419,09:00:00,630119.0003458913 5807102.019045829,629401.457820961 5809413.341770453
3,1033909,09:00:00,627217.3355998315 5805188.024605306,627313.6303041776 5804683.771269505
4,1122860,09:00:00,633434.4205306872 5800327.508477634,632524.5993621239 5801905.183290979
...,...,...,...,...
46857,1094097,11:00:00,630629.0367625962 5801999.458889815,630982.3969120757 5802597.144638936
46858,1084155,11:00:00,625513.0253870577 5800572.022440629,627467.7036338786 5801350.716498636
46859,1236326,11:00:00,601101.0862328823 5779515.337970847,600307.3043922537 5778801.142549095
46860,1009137,11:00:00,639980.5338344915 5847537.867287946,629898.2484026412 5805817.793417794


In [110]:
len(Input)

46862

In [104]:
def missingData(inputDataframe, listID_in, listID_out):
    missing = diff(listID_in, listID_out)
    indexUser = [Input.index[Input['ID'] == i].tolist() for i in missing]  # index of missing user
    indexUser = list(itertools.chain.from_iterable(indexUser))
    return Input.iloc[indexUser]

In [119]:
df_missing = missingData(Input, listID_in, listID_out)
df_missing

,ID,DEPARTURE,ORIGIN,DESTINATION
9930,917515,09:18:14,613738.9128652506 5818141.26617263,629243.7397205335 5804437.454238096
43780,917515,10:53:12,629243.7397205335 5804437.454238096,613738.9128652506 5818141.26617263
13668,983051,09:25:17,630285.1689666857 5823722.328112848,631435.8993262072 5823049.13385241
4350,1048595,09:07:52,622015.0947725042 5773928.925534364,626851.9692397028 5803852.636913614
31168,589845,10:24:39,691913.3587404573 5762532.257194359,687412.3284794632 5761766.715623426
...,...,...,...,...
9813,786404,09:18:01,639430.1537932879 5778427.223724365,642672.2692429221 5773995.791650763
25632,786404,10:09:22,642672.2692429221 5773995.791650763,639430.1537932879 5778427.223724365
11929,1114103,09:21:59,634526.9620972774 5798128.515459549,607382.294792387 5756226.626322996
15049,917501,09:28:00,612791.2251259275 5818655.7754812995,629088.1051609886 5803890.015116696


In [109]:
#df.sort_values(by = 'ID')

In [15]:
def bounding(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    l = O + D  # Concatenate 2 2D lists
    coord = np.array([x.split() for x in l])
    Xmin = min(float(x[0]) for x in coord)
    Ymin = min(float(x[1]) for x in coord)
    Xmax = max(float(x[0]) for x in coord)
    Ymax = max(float(x[1]) for x in coord)
    return Xmin, Xmax, Ymin, Ymax

In [4]:
# Highway coordinates
x, y, zone, ut = utm.from_latlon(52.33897076764408, 4.8261031256321365)
(x,y)

(624408.5720968425, 5800310.2899306165)

In [133]:
def in_out_city(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    ID = df.ID.tolist()  ## no use
    array_O = np.array([x.split() for x in O])
    array_D = np.array([x.split() for x in D])
    coord_O = array_O.astype(float)
    coord_D = array_D.astype(float)
    # Bounding of city Amsterdam
    (xmin, ymin, xmax, ymax) = (621225.6754145394, 5795122.850697235, 635225.6754145394, 5810622.850697235)
    In = lambda x, y : x <= xmax and x >= xmin and y <= ymax and y>= ymin  # if between xmin and xmax
    Out = lambda x, y : x > xmax or x < xmin or y > ymax or y < ymin   # if not
    (in_in, in_out, out_in, out_out) = (0, 0, 0, 0)
    for i in range(len(O)):
        if In(coord_O[i][0], coord_O[i][1]) & In(coord_D[i][0], coord_D[i][1]):
            in_in+=1
        elif In(coord_O[i][0], coord_O[i][1]) & Out(coord_D[i][0], coord_D[i][1]):
            in_out+=1
        elif Out(coord_O[i][0], coord_O[i][1]) & In(coord_D[i][0], coord_D[i][1]):
            out_in+=1
        elif Out(coord_O[i][0], coord_O[i][1]) & Out(coord_D[i][0], coord_D[i][1]):
            out_out+=1
    per_in_in = (in_in / len(df))*100
    per_in_out = (in_out / len(df))*100
    per_out_in = (out_in / len(df))*100
    per_out_out = (out_out / len(df))*100
    sum_out = in_out + out_in + out_out
    return per_in_in, per_in_out, per_out_in, per_out_out, sum_out

In [134]:
(a,b,c,d,e) = in_out_city(Input)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 55% in_out_city : 11% out_in_city : 10% out_out_city : 24% sum_out : 21006


In [135]:
(a,b,c,d,e) = in_out_city(df_missing)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 0% in_out_city : 19% out_in_city : 19% out_out_city : 62% sum_out : 15250


In [101]:
##
O = Input.ORIGIN.tolist()
coord_O = np.array([x.split() for x in O])
y = coord_O.astype(float)
y[0][1]

5804534.655578803

In [18]:
##
x1,x2,y1,y2 = bounding(Input)
print('Xmin :', x1, 'Ymin :', x2, 'Xmax :', y1, 'Ymax :', y2)

Xmin : 565468.5380879932 Ymin : 773275.7797587274 Xmax : 5541622.850697235 Ymax : 5902899.484527369


In [19]:
O = Input.ORIGIN.tolist()  
D = Input.DESTINATION.tolist()
l = O + D  # Concatenate 2 2D lists
coord = np.array([x.split() for x in l])
Xmin = min(float(x[0]) for x in coord)
Ymin = min(float(x[1]) for x in coord)
Xmax = max(float(x[0]) for x in coord)
Ymax = max(float(x[1]) for x in coord)
#print('Xmin :', Xmin, 'Ymin :', Ymin, 'Xmax :', Xmax, 'Ymax :', Ymax)

Xmin : 565468.5380879932 Ymin : 5541622.850697235 Xmax : 773275.7797587274 Ymax : 5902899.484527369


In [138]:
num2 = random.uniform(621225.6754145394, 635225.6754145394)
print(num2)

621708.9514420278


In [141]:
## Modifie the departure or destination coordinate
def all_in_city(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    l_O = [x.split() for x in O]
    l_D = [x.split() for x in D]
    (xmin, ymin, xmax, ymax) = (621225.6754145394, 5795122.850697235, 635225.6754145394, 5810622.850697235)
    In = lambda x, y : x <= xmax and x >= xmin and y <= ymax and y>= ymin  # if between xmin and xmax
    Out = lambda x, y : x > xmax or x < xmin or y > ymax or y < ymin   # if not
    in_x = lambda x: str(random.uniform(xmin, xmax)) 
    in_y = lambda y: str(random.uniform(ymin, ymax)) 
    ## Find the coordinates of the highway entrance
    for i in range(len(O)):
        if not(In(float(l_O[i][0]), float(l_O[i][1])) & In(float(l_D[i][0]), float(l_D[i][1]))):
            if In(float(l_O[i][0]), float(l_O[i][1])) & Out(float(l_D[i][0]), float(l_D[i][1])):

            elif Out(float(l_O[i][0]), float(l_O[i][1])) & In(float(l_D[i][0]), float(l_D[i][1])):

            else:
            # supprime tout 

IndentationError: expected an indented block (1711961521.py, line 16)

In [140]:
O = Input.ORIGIN.tolist()
[x.split() for x in O]

[['624582.3745792772', '5804534.655578803'],
 ['612373.9501705638', '5801667.450917384'],
 ['630119.0003458913', '5807102.019045829'],
 ['627217.3355998315', '5805188.024605306'],
 ['633434.4205306872', '5800327.508477634'],
 ['638936.5085865267', '5820127.096951594'],
 ['626255.1136157031', '5803246.704181978'],
 ['633156.624022941', '5806290.24244078'],
 ['626607.0753754578', '5804374.9727834035'],
 ['627998.259583113', '5805387.702510312'],
 ['629011.2147521778', '5804799.604854093'],
 ['651823.6499216193', '5795769.941772221'],
 ['638124.2985466479', '5813252.449151918'],
 ['612028.4769526601', '5808589.190537517'],
 ['608057.6107962361', '5795614.132942132'],
 ['628769.786846441', '5801423.179996274'],
 ['611886.8291091116', '5815126.131272016'],
 ['639220.7828297943', '5797503.276700023'],
 ['639421.4630223699', '5797999.277972572'],
 ['634611.5032868218', '5818968.898751193'],
 ['627941.4103353849', '5804535.854945883'],
 ['633082.1036581361', '5806463.998059905'],
 ['635276.465

# Indicateurs pour les véhicules

In [68]:
input_veh = os.getcwd() + "/OUTPUTS_9h_11h/veh.csv"
veh = pd.read_csv(input_veh, delimiter = ";")
veh

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
0,09:00:00.00,175,Car,NaN,NaN,13.80,SERVING,0.00,1031082.0
1,09:00:00.00,174,Car,m46324645 m46329700,NaN,13.80,SERVING,0.00,1067560.0
2,09:00:00.00,173,Car,m46448801 m46447993,NaN,13.80,SERVING,0.00,1044279.0
3,09:00:00.00,172,Car,m4795047522 m949979531,NaN,13.80,SERVING,0.00,1081188.0
4,09:00:00.00,171,Car,m2386931641 m46087374,NaN,13.80,SERVING,0.00,1116804.0
...,...,...,...,...,...,...,...,...,...
244844,11:00:00.00,30348,Car,m46386907 m943792454,630080.997 5804095.810,10.73,SERVING,643.78,1004591.0
244845,11:00:00.00,30350,Car,m3301116661 m46361992,627968.949 5803378.766,10.73,SERVING,643.78,1067723.0
244846,11:00:00.00,30351,Car,m26371852 m46339660,626936.898 5802793.960,10.73,SERVING,643.78,1104106.0
244847,11:00:00.00,30354,Car,m46412389 m46409941,623934.541 5804520.039,10.73,SERVING,643.78,1017722.0


## Distance

In [45]:
dist = veh.groupby(by = 'ID')[['DISTANCE']].agg(['sum','min','max'])
dist

DISTANCE               
            sum  min       max
ID                            
0      3462.326  0.0  1404.666
1      9864.094  0.0  3036.654
2         0.000  0.0     0.000
3      1666.669  0.0   979.389
4         0.000  0.0     0.000
...         ...  ...       ...
30351   643.780  0.0   643.780
30352     0.000  0.0     0.000
30353     0.000  0.0     0.000
30354   643.780  0.0   643.780
30355   643.780  0.0   643.780

[30356 rows x 3 columns]

## Speed

In [44]:
speed = veh.groupby(by = 'ID')[['SPEED']].agg(['mean','min','max'])
speed

SPEED              
            mean     min   max
ID                            
0      11.984500  11.298  13.8
1      11.719500  11.157  13.8
2      13.800000  13.800  13.8
3      12.213333  11.385  13.8
4      13.800000  13.800  13.8
...          ...     ...   ...
30351  12.265000  10.730  13.8
30352  13.800000  13.800  13.8
30353  13.800000  13.800  13.8
30354  12.265000  10.730  13.8
30355  12.265000  10.730  13.8

[30356 rows x 3 columns]

## Travel time

In [69]:
time = veh['TIME'].tolist()
#[lambda x :( float(x[:2])*3600 + float(x[3:5])*60 + float(x[6:])) for x in time ]
turn_to_second = lambda x : float(x[:2])*3600 + float(x[3:5])*60 + float(x[6:]) - 9*3600  # Turn to second
for_all = lambda l, f : [f(i) for i in l]
travel_time = for_all(time, turn_to_second)
travel_time

39600.0

In [70]:
veh['TIME'] = travel_time
veh

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
0,32400.0,175,Car,NaN,NaN,13.80,SERVING,0.00,1031082.0
1,32400.0,174,Car,m46324645 m46329700,NaN,13.80,SERVING,0.00,1067560.0
2,32400.0,173,Car,m46448801 m46447993,NaN,13.80,SERVING,0.00,1044279.0
3,32400.0,172,Car,m4795047522 m949979531,NaN,13.80,SERVING,0.00,1081188.0
4,32400.0,171,Car,m2386931641 m46087374,NaN,13.80,SERVING,0.00,1116804.0
...,...,...,...,...,...,...,...,...,...
244844,39600.0,30348,Car,m46386907 m943792454,630080.997 5804095.810,10.73,SERVING,643.78,1004591.0
244845,39600.0,30350,Car,m3301116661 m46361992,627968.949 5803378.766,10.73,SERVING,643.78,1067723.0
244846,39600.0,30351,Car,m26371852 m46339660,626936.898 5802793.960,10.73,SERVING,643.78,1104106.0
244847,39600.0,30354,Car,m46412389 m46409941,623934.541 5804520.039,10.73,SERVING,643.78,1017722.0


In [74]:
veh.groupby(by = 'ID').sum()

,TIME,SPEED,DISTANCE,PASSENGERS
ID,,,,
0,129960.0,47.938,3462.326,3101727.0
1,195300.0,70.317,9864.094,5232675.0
2,32400.0,13.800,0.000,1035359.0
3,97380.0,36.640,1666.669,2118386.0
4,32400.0,13.800,0.000,1086164.0
...,...,...,...,...
30351,79140.0,24.530,643.780,2208212.0
30352,39540.0,13.800,0.000,1091688.0
30353,39540.0,13.800,0.000,1111118.0


## 